# 1. Setup and Get Data

### 1.1 Install Dependencies and Setup

yolo

In [4]:
import os
import json

#  images and JSON labels
data_folder = '/home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/data/'

# to convert JSON labels to YOLO format
def convert_to_yolo_format(json_data, img_width, img_height):
    yolo_labels = []
    for shape in json_data['shapes']:
        if shape['label'] == 'face':
            points = shape['points']
            
            x1, y1 = points[0]
            x2, y2 = points[1]
            
            
            x_center = (x1 + x2) / 2 / img_width
            y_center = (y1 + y2) / 2 / img_height
            width = (x2 - x1) / img_width
            height = (y2 - y1) / img_height
            
            # Append class_id (face = 0) and normalized coordinates
            yolo_labels.append(f"0 {x_center} {y_center} {width} {height}")
    return yolo_labels

# Convert all JSON files to YOLO format
for filename in os.listdir(data_folder):
    if filename.endswith('.json'):
        json_path = os.path.join(data_folder, filename)
        with open(json_path, 'r') as f:
            data = json.load(f)
        
        img_width = data['imageWidth']
        img_height = data['imageHeight']
        yolo_labels = convert_to_yolo_format(data, img_width, img_height)
        
        # Save YOLO labels to a .txt file with the same name as the image
        txt_filename = filename.replace('.json', '.txt')
        txt_path = os.path.join(data_folder, txt_filename)
        with open(txt_path, 'w') as f:
            f.write('\n'.join(yolo_labels))


In [5]:
!pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 876.6/876.6 kB 45.4 kB/s eta 0:00:00a 0:00:03
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 494.4 kB/s eta 0:00:0000:0100:01
ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json


In [7]:
import os
import json

def convert_to_yolo_format(json_file, img_width, img_height):
    """Converts JSON bounding boxes to YOLO format."""
    yolo_data = []
    with open(json_file, 'r') as f:
        data = json.load(f)
        shapes = data['shapes']
        
        for shape in shapes:
            label = 0  # Assuming 'face' is the only class, thus class_id is 0
            points = shape['points']
            x_min, y_min = points[0]
            x_max, y_max = points[1]

            # Compute YOLO format
            x_center = (x_min + x_max) / 2.0 / img_width
            y_center = (y_min + y_max) / 2.0 / img_height
            width = (x_max - x_min) / img_width
            height = (y_max - y_min) / img_height

            yolo_data.append(f"{label} {x_center} {y_center} {width} {height}")
    
    return yolo_data

def convert_json_to_txt(label_dir, output_dir, img_dir):
    """Converts all JSON label files in a directory to YOLO .txt format."""
    for json_filename in os.listdir(label_dir):
        if json_filename.endswith('.json'):
            json_path = os.path.join(label_dir, json_filename)
            with open(json_path, 'r') as f:
                data = json.load(f)
                image_path = os.path.join(img_dir, os.path.basename(data['imagePath']))
                img_width = data['imageWidth']
                img_height = data['imageHeight']
                
                yolo_data = convert_to_yolo_format(json_path, img_width, img_height)
                
                # Write YOLO label file
                output_txt_path = os.path.join(output_dir, json_filename.replace('.json', '.txt'))
                with open(output_txt_path, 'w') as txt_file:
                    txt_file.write("\n".join(yolo_data))

# Set your paths
label_dir_train = '/home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/data/labels/train'
label_dir_val = '/home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/data/labels/val'
img_dir_train = '/home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/data/images/train'
img_dir_val = '/home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/data/images/val'
output_dir_train = label_dir_train  # Labels will be saved in the same folder
output_dir_val = label_dir_val

# Convert train and val sets
convert_json_to_txt(label_dir_train, output_dir_train, img_dir_train)
convert_json_to_txt(label_dir_val, output_dir_val, img_dir_val)


In [10]:
from ultralytics import YOLO

# Load a YOLOv8n model, available variants are yolov8n, yolov8s, yolov8m, yolov8l, yolov8x
model = YOLO('yolov8n.pt')  # Download and load the pre-trained YOLOv8n model

# Train the model
model.train(data='/home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data//data/config.yaml', epochs=100, imgsz=640, batch=8)


New https://pypi.org/project/ultralytics/8.3.20 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.19 🚀 Python-3.11.10 torch-2.2.2+cu121 CPU (Intel Core(TM) i5-8250U 1.60GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data//data/config.yaml, epochs=100, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=N

100%|██████████| 755k/755k [00:01<00:00, 544kB/s] 

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 22        [15, 18, 21]  1    751507  ultralytics.nn.modules.head.Detect           [1, [64, 128, 256]]           
Model summary: 225 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
TensorBoard: Start with 'tensorboard --logdir runs/detect/train2', view at http://localhost:6006/
Freezing layer 'model.22.dfl.conv.weight'


train: Scanning /home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/data/labels/train... 52 images, 8 backgrounds, 0 corrupt: 100%|██████████| 60/60 [00:00<00:00, 1548.60it/s]

...

                   all         15         15      0.996          1      0.995      0.729



100 epochs completed in 0.712 hours.
Optimizer stripped from runs/detect/train2/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train2/weights/best.pt, 6.2MB

Validating runs/detect/train2/weights/best.pt...
Ultralytics 8.3.19 🚀 Python-3.11.10 torch-2.2.2+cu121 CPU (Intel Core(TM) i5-8250U 1.60GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:01<00:00,  1.96s/it]


                   all         15         15          1      0.996      0.995      0.778
Speed: 2.1ms preprocess, 114.5ms inference, 0.0ms loss, 3.2ms postprocess per image
Results saved to runs/detect/train2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x784b84fc3910>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [11]:
# Validate the model
metrics = model.val()
print(metrics)


Ultralytics 8.3.19 🚀 Python-3.11.10 torch-2.2.2+cu121 CPU (Intel Core(TM) i5-8250U 1.60GHz)
Model summary (fused): 168 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/data/labels/val.cache... 15 images, 0 backgrounds, 0 corrupt: 100%|██████████| 15/15 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.04s/it]


                   all         15         15          1      0.996      0.995      0.778
Speed: 3.4ms preprocess, 123.0ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to runs/detect/train22
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x784b90b56510>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.

In [27]:
import json

results = model('/home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/test/images/dc30cbd6-8d90-11ef-b7d1-a683e7996e5e.jpg')
results[0].show() 
predictions = []
group_id_counter = 1

for pred in results[0].boxes:
    bbox = pred.xyxy[0].cpu().numpy().tolist()

    group_id = getattr(pred, 'group_id', None)

    if group_id is None:
        group_id = group_id_counter
        group_id_counter += 1

    predictions.append({
        "label": "face",
        "points": [
            [bbox[0], bbox[1]],
            [bbox[2], bbox[3]]
        ],
        "group_id": group_id,
        "description": "",
        "shape_type": "rectangle",
        "flags": {}
    })

json_output = json.dumps(predictions, indent=4)

output_path =  '/home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/data/your_output.json'

with open(output_path, 'w') as outfile:
    outfile.write(json_output)

print(f"JSON file saved at {json_output}")


image 1/1 /home/waqar/scrapper/Aziz_facedector/FaceRecognition2/data/test/images/dc30cbd6-8d90-11ef-b7d1-a683e7996e5e.jpg: 480x640 1 face, 244.4ms
Speed: 34.4ms preprocess, 244.4ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)
JSON file saved at [
    {
        "label": "face",
        "points": [
            [
                230.53607177734375,
                75.81787109375
            ],
            [
                448.3497314453125,
                411.35809326171875
            ]
        ],
        "group_id": 1,
        "description": "",
        "shape_type": "rectangle",
        "flags": {}
    }
]
